In [ ]:
from google.colab import drive
drive.mount('/content/drive')

To install dependencies

Note: After installation, please restart the runtime if it is required.

In [ ]:
!pip install ultralytics
!pip install av

Milestone 1 (Player detection using the YOLOv8 models trained on COCO dataset)

In [ ]:
import sys
root_path = '/content/drive/MyDrive/basketball-broadcaster'
sys.path.append(root_path)

import os
os.chdir(root_path)

import cv2
from yolov8 import YOLOv8
import time
import av
import numpy as np
from PIL import Image

# We can use 5 kinds of YOLOv8 models
player_detector = YOLOv8('yolov8m.pt') # 'yolov8n.pt', 'yolov8s.pt', 'yolov8m.pt', 'yolov8l.pt' or 'yolov8x.pt'

input_ = av.open("videos/test1_clip.mp4")
output = av.open("output.mp4", "w")

# Make an output video stream using the input video stream as a template
in_video_stream = input_.streams.video[0]
out_video_stream = output.add_stream(codec_name="h264",
                                     width=in_video_stream.width,
                                     height=in_video_stream.height,
                                     rate=int(in_video_stream.average_rate),
                                     bit_rate = in_video_stream.bit_rate,
                                     pix_fmt=in_video_stream.pix_fmt)

# Make an output audio stream using the input audio stream as a template
in_audio_stream = input_.streams.audio[0]
out_audio_stream = output.add_stream(template=in_audio_stream)


for packet in input_.demux():
    # We need to skip the "flushing" packets that `demux` generates.
    if packet.dts is None:
        continue

    if packet.stream == in_video_stream:
        for frame in packet.decode():
            # To get an input frame
            image = frame.to_image()
            image = np.asarray(image)
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            # To process the input frame
            boxes, scores, class_ids = player_detector.detect(image)
            image = player_detector.draw_detection(image, boxes, scores, class_ids)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = Image.fromarray(image)

            output_frame = av.VideoFrame.from_image(image)
            output_frame.pts = None
            for output_packet in out_video_stream.encode(output_frame):
                # Write the packets to the container
                output_packet.stream = out_video_stream
                output.mux(output_packet)
    elif packet.stream == in_audio_stream:
        packet.stream = out_audio_stream
        output.mux(packet)

input_.close()
output.close()